In [1]:
# You need to install this dependencies

# langchain
# langchain-community
# langchain_core
# langchain-openai langchainhub
# openai==1.11.0
# keras==3.1.1
# tensorflow==2.16.1
# matplotlib==3.8.3
# tensorflow_hub==0.16.1
# sentence-transformers


import pandas as pd
import numpy as np

import os
import dotenv
from langchain_core.prompts import PromptTemplate
from langchain import LLMChain
from langchain_openai import OpenAI
from langchain.callbacks import get_openai_callback
import json

template = """I'll pass you a cocktail name and its ingredients with a\
    brandname and you need to return in a json, the base liquor and its\
    ingredients without measures and for example:\
    cocktail:West Coast Punch
    ingredients: 1.5 oz Junipero Gin, .75 oz Pineapple Juice,\
    .5 oz Monin Strawberry Puree, 6-7  Mint Leaves, .5 oz Lemon Sour, 1 oz Lemon Lime Soda
    return:
    
   {{"base_liquor":"gin","ingredients":"Junipero Gin, Pineapple Juice,\
    Monin Strawberry Puree, Mint Leaves, Lemon Sour, Lemon Lime Soda"}}
    
    
    cocktail name: {cocktail}\
    list of ingredients: {ingredients}
"""

print(os.getcwd())

/Users/rafaeldavila/Documents/Duke/Sem2/Cloud/final_project/MIDS_MIX/00_code


In [19]:
os.listdir("../../data")

['hotaling_cocktails_Cocktails.csv']

In [21]:
data.shape

(687, 9)

In [24]:
data = pd.read_csv("../../data/hotaling_cocktails_Cocktails.csv")
responses = []
counter = 0

for _, row in data.iterrows():

    prompt = PromptTemplate(
        input_variables=["cocktail", "ingredients"], template=template
    )
    llm_gpt3_5_chain = LLMChain(prompt=prompt, llm=OpenAI())

    input_params = {
        "cocktail": row["Cocktail Name"],
        "ingredients": row["Ingredients"],
        "temperature": 0,
    }
    response = llm_gpt3_5_chain.invoke(input_params)
    responses.append(response["text"])

    counter += 1
    if counter % 50 == 0:
        print(f"Number: {counter}")

Number: 50
Number: 100
Number: 150
Number: 200
Number: 250
Number: 300
Number: 350
Number: 400
Number: 450
Number: 500
Number: 550
Number: 600
Number: 650


In [27]:
import pandas as pd
import re


def get_base_liquor(responses):
    base_liquor = []
    for response in responses:
        match = re.search(r'"base_liquor":"(.*?)"', response)
        if match:
            base_liquor.append(match.group(1))
        else:
            base_liquor.append(None)
    return base_liquor


def get_ingredients(responses):
    ingredients = []
    for response in responses:
        match = re.search(r'"ingredients":"(.*?)"', response)
        if match:
            ingredients.append(match.group(1))
        else:
            ingredients.append(None)
    return ingredients


data["base_liquor"] = get_base_liquor(responses)
data["cleaned_ingredients"] = get_ingredients(responses)

In [28]:
data.sample(10)

,Cocktail Name,Bartender,Bar/Company,Location,Ingredients,Garnish,Glassware,Preparation,Notes,base_liquor,cleaned_ingredients
10,Happiness,Danny Louie,Mister Jiu's,San Francisco,"1.5 oz Jasmine Tea/Rice-Infused Junipero Gin*,...",Edible Flower,Coupe,"Add all ingredients in a cocktail shaker, shak...",Junipero Gin 20th Anniversary Signature Cocktail,gin,"Jasmine Tea, Rice, Sour Apple Liqueur, Lime, H..."
456,"Eat, Drink and Be Cherry",Paul Taylor,Cherry Blossom PUB,Washington DC,".75 oz Nikka Coffey Grain Whisky, .75 oz Luxar...",3 Cherries Picked,Bamboo Highball Glass,"Shake everything except for the soda water, st...",NaN,whisky,"Nikka Coffey Grain Whisky, Luxardo Sangue Morl..."
274,Prickly Pear Margarita,Edgar & Joselino Solis,Matador,San Francisco,"2 oz Chinaco Blanco Tequila, 1 oz Lime Juice, ...",Mint Sprig,Margarita Glass,Mint salt on the rim. Garnish with mint sprig.,NaN,tequila,"Chinaco Blanco Tequila, Lime Juice, Prickly Pe..."
517,Pisco Punch (Adaptation by The Comstock Saloon),NaN,Comstock Saloon,San Francisco,"1.5 oz BarSol Selecto Italia Pisco, .75 oz Sma...",Lemon Peel,Flip Glass,"Shake, strain into a chilled flip glass with o...",Featured on Speed Rack 2015 SF Speed City Take...,pisco,"BarSol Selecto Italia Pisco, Small Hand Foods ..."
314,Sage & Sound,Brian Means,Michael Mina Restaurant Group,San Francisco,"1.5 oz Junipero Gin, 1 oz White Port, .75 oz L...",Whole Star Anise,Coupe,Muddle the sage in a mixing tin. Add the remai...,Junipero Gin 20th Anniversary Signature Cocktail,gin,"Junipero Gin, White Port, Lemon Juice, Honey C..."
24,The King's Autumn Cobbler,NaN,NaN,NaN,"1 oz King's Ginger Liqueur, .25 oz Briottet Cr...","3 Lemon Wedges, 3 Blackberries",NaN,Add all ingredients over ice. Stir with a bars...,NaN,ginger liqueur,"King's Ginger Liqueur, Briottet Creme de Mure,..."
624,Nigamono,John deBarry,Momofuku,NaN,"2 oz Nikka Coffey Grain Whisky, .75 oz Maurin ...",Luxardo Cherries,Double Old Fashioned Glass,Stir. Use large ice cube.,NaN,whisky,"Nikka Coffey Grain Whisky, Maurin Quina, Luxar..."
305,Sabotage,William Golde,NaN,NaN,"2 oz Breck Bourbon, .5 oz Dancing Pines Black ...","Burnt orange, Thyme sprig",Rocks Glass,"Muddle sage and Thyme. Stirred, over ice in si...",NaN,bourbon,"Breck Bourbon, Dancing Pines Black Walnut Bour..."
652,Speyside Highball,NaN,NaN,NaN,"1.5 oz Glenrothes Vintage Reserve, 1 oz Tuaca,...",Cucumber,Highball Glass,NaN,NaN,scotch,"Glenrothes Vintage Reserve, Tuaca, Lemon Juice..."
177,Elgin Elixir,Alan Kropf,NaN,San Francisco,"1 oz BenRiach Single Malt 12 YO, 1 oz C. Droui...",NaN,Rocks Glass,Combine ingredients with ice in a mixing glass...,NaN,whiskey,"BenRiach Single Malt 12 YO, C. Drouin Calva..."


In [38]:
data["base_liquor"] = [
    i.lower() if isinstance(i, str) else i for i in data["base_liquor"]
]
frequency_table = data["base_liquor"].value_counts()

In [42]:
data.to_csv("../../data/final_data.csv", index=False)

In [45]:
top_20_base_liquor = frequency_table[:20].index
filtered_data = data[data["base_liquor"].isin(top_20_base_liquor)]
# filtered_data.to_csv("../../data/filtered_data.csv", index=False)

In [46]:
filtered_data.shape

(604, 11)

# Adding Embeddings

In [22]:
from sentence_transformers import SentenceTransformer
import pandas as pd

data = pd.read_csv("../10_data/filtered_data.csv")

In [51]:
data.head()

,ID,name,base_liquor,cleaned_ingredients,embeddings
0,ID1,Flor de Amaras,mezcal,"Mezcal, Hibiscus Simple Syrup, Lime Juice, Sod...","[-0.050698716, 0.0269325, 0.00097512745, 0.021..."
1,ID2,The Happy Place,gin,"Junipero Gin, House-made Cranberry Syrup, Lemo...","[-0.10407846, -0.06247114, -0.006959201, -0.01..."
2,ID3,Bon Voyage Pisco Punch,pisco,"BarSol Selecto Italia Pisco, Lemon Juice, Pine...","[-0.063485615, -0.013461153, -0.0008486637, 0...."
3,ID4,Still Life of a Pineapple,pisco,"BarSol Primero Quebranta Pisco, Dry Vermouth, ...","[-0.02595515, -0.06881333, 0.0041712066, -0.02..."
4,ID5,The Bittered Valley,pisco,"Luxardo Maraschino Liqueur, Acid phosphate, Ba...","[-0.025188107, -0.08992694, 0.027692243, 0.011..."


In [23]:
model = SentenceTransformer("all-MiniLM-L6-v2")

# Sentences are encoded by calling model.encode()
# You need to rerun the embeddings because when saved as csv it deletes the commas
# in the embeddings
embeddings = [model.encode(v) for v in data["cleaned_ingredients"]]

In [25]:
data["embeddings"] = embeddings

In [54]:
len(embeddings[0])

384

In [11]:
# data.to_csv("../10_data/filtered_data.csv", index=False)

In [52]:
import ast

vectors = []
for _, row in data.iterrows():
    vectors.append(
        {
            "id": row["ID"],
            "values": row["embeddings"],
            "metadata": {
                "name": row["name"],
                "base": row["base_liquor"],
                "ingredients": row["cleaned_ingredients"],
            },
        }
    )

In [58]:
from pinecone import Pinecone, ServerlessSpec, PodSpec
import os

pc = Pinecone(api_key=os.getenv("PINECONE_API_KEY"))

try:
    pc.delete_index("midsmix")
except:
    print("index already deleted")

pc.create_index(
    name="midsmix",
    dimension=384,
    metric="cosine",
    spec=ServerlessSpec(cloud="aws", region="us-east-1"),
)

index = pc.Index("midsmix")

index.upsert(vectors)

index already deleted


{'upserted_count': 603}

In [38]:
import random


def return_image(ingredients=None, random_retrieve=True):
    """This function returns the url of the image
    using a random selection or using the closest image


    Args:
        base_liquor (str): base liquour among 20 different
        random (bool, optional): Is the return random or not. Defaults to True.
    """
    url_drinks = {
        "tequila": "https://www.lanaval.com.mx/115615-product_default/tequila-don-julio-70-700-ml.jpg",
        "whiskey": "https://bouquetdebarrica.com/cdn/shop/products/WoodfordReserveMain.jpg?v=1659570654",
        "beer": "https://cocktail-recommendations.s3.amazonaws.com/cocktail-pictures/Hi-WireBrewery/MountainWater.jpg",
        "vodka": "https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcSg9YSJgWS8UFXkJlmnWkBx5c9Ynx4MjhRP4k7WqFun7A&s",
    }

    if random_retrieve:
        random_keys = random.sample(list(url_drinks.keys()), 3)
        random_values = [url_drinks[key] for key in random_keys]
        return random_values

    else:
        # Using lambda to retrieve the info.
        return None

In [69]:
return_image(ingredients=None, random_retrieve=True)

['https://cocktail-recommendations.s3.amazonaws.com/cocktail-pictures/Hi-WireBrewery/MountainWater.jpg',
 'https://www.lanaval.com.mx/115615-product_default/tequila-don-julio-70-700-ml.jpg',
 'https://bouquetdebarrica.com/cdn/shop/products/WoodfordReserveMain.jpg?v=1659570654']

## Retrieving three random veberages

In [ ]:
index.fetch(ids=["id"])

In [59]:
def retrieve_random_coktails():

    three_ids = random.sample(range(1, 604), 3)

    three_ids = ["ID" + str(i) for i in three_ids]

    result = index.fetch(ids=three_ids)

    return result["vectors"]


vectors = retrieve_random_coktails()

In [64]:
{"drink1": "gin", "drink1": "tequila"}

{'ID175': {'id': 'ID175',
  'metadata': {'base': 'amaretto',
               'ingredients': 'Luxardo Amaretto di Saschira, Rye Whiskey, Lemon '
                              'Juice, Angostura Bitters, Egg White',
               'name': 'Luxardo Sour'},
  'values': [-0.0413903482,
             -0.0318790115,
             -0.0227916725,
             0.0665141791,
             -0.00750292,
             0.0933137685,
             0.0252350867,
             0.0563966446,
             -0.0172521826,
             -0.0616632476,
             0.0211636871,
             0.0104164099,
             -0.109939478,
             -0.0458480455,
             -0.0455773398,
             -0.0223323945,
             0.0754728615,
             0.0357578062,
             -0.0507041439,
             -0.0400283486,
             0.0223393813,
             -0.0809861869,
             0.0542032272,
             0.0957299173,
             -0.0121225184,
             0.0925310925,
             0.0530515164,
        